In [98]:
import torch as th

from torchvision import transforms

import matplotlib.pyplot as plt

In [ ]:
from src.dataloader import customDataloader

In [100]:
train_loader = customDataloader(train=True,  transform=None, batch_size=64, shuffle=True)
test_loader  = customDataloader(train=False, transform=None, batch_size=64, shuffle=True)